# Host-guest usage with stk

by: Andrew Tarzia

### Update 25/02/24

Installation of py3Dmol using `pip install py3Dmol` is required.

#### Using latest version of `SpinDry` and `stk`.

- Changes in `SpinDry.Molecule` means we now use `spd.Molecule()` not `spd.Molecule.init()`
- `spd.Spinner.get_conformers` should now take `spd.Supramolecule`, which we initialise from components.
- To then access those components, you must do that through the `Supramolecule` interface.
- Finally, we had to change to using the `stk.complex.Guest` interface.
- See https://stk.readthedocs.io/en/stable/_autosummary/stk.host_guest.Complex.html
- See https://github.com/andrewtarzia/SpinDry/blob/main/examples/

# Imports 

In [1]:
import sys
sys.path.append('/usr/local/lib/python3.7/site-packages/')

from rdkit import Chem 
from rdkit.Chem import AllChem as rdkit
from collections import defaultdict
from rdkit.Chem import rdFMCS
from rdkit.Chem import Draw
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import rdDistGeom
IPythonConsole.ipython_3d = True

import py3Dmol
from IPython.display import Image
import matplotlib.pyplot as plt
import subprocess
import time
import stk
import stko
import spindry as spd
%matplotlib inline

# Some useful functions 

In [2]:
def show_stk_mol(stk_mol):
    data = rdkit.MolToMolBlock(stk_mol.to_rdkit_mol())
    p = py3Dmol.view(
        data=data,
        style={'stick':{'colorscheme':'cyanCarbon'}}, 
        width=400,
        height=400,
    )
    p.setBackgroundColor('0xeeeeee')
    p.zoomTo()
    p.show()

Working:

In [3]:
# Produce a Pd+2 atom with 4 functional groups.
palladium_atom = stk.BuildingBlock(
    smiles='[Pd+2]',
    functional_groups=(
        stk.SingleAtom(stk.Pd(0, charge=2))
        for i in range(4)
    ),
    position_matrix=[[0., 0., 0.]],
)

# Build a building block with two functional groups using
# the SmartsFunctionalGroupFactory.
bb1 = stk.BuildingBlock(
    smiles=(
        'C1=NC=CC(C2=CC=CC(C3=CC=NC=C3)=C2)=C1'
    ),
    functional_groups=[
        stk.SmartsFunctionalGroupFactory(
            smarts='[#6]~[#7X2]~[#6]',
            bonders=(1, ),
            deleters=(),
        ),
    ],
)

cage1 = stk.ConstructedMolecule(
    stk.cage.M6L12Cube(
        building_blocks=(palladium_atom, bb1),
        # Ensure that bonds between the GenericFunctionalGroups
        # of the ligand and the SingleAtom functional groups
        # of the metal are dative.
        reaction_factory=stk.DativeReactionFactory(
            stk.GenericReactionFactory(
                bond_orders={
                    frozenset({
                        stk.GenericFunctionalGroup,
                        stk.SingleAtom
                    }): 9
                }
            )
        ),
        optimizer=stk.MCHammer(num_steps=2000),
    )
)

In [4]:
show_stk_mol(cage1)

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [5]:
stk_guest = stk.BuildingBlock('C1C(O[H])C2C(C=CC(=C2)CP([H])[H])CC1')
show_stk_mol(stk_guest)

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [6]:
cage_atoms = [
    (atom.get_id(), atom.__class__.__name__)
    for atom in cage1.get_atoms()
]
stk_mol_atoms = [
    (atom.get_id(), atom.__class__.__name__)
    for atom in stk_guest.get_atoms()
]

In [7]:
host = spd.Molecule(
    atoms=(
        spd.Atom(id=i[0], element_string=i[1])
        for i in cage_atoms
    ),
    bonds=(),
    position_matrix=cage1.get_position_matrix(),
)
guest = spd.Molecule(
    atoms=(
        spd.Atom(id=i[0], element_string=i[1])
        for i in stk_mol_atoms
    ),
    bonds=(),
    position_matrix=stk_guest.get_position_matrix(),
)

In [8]:
print(host, guest)

<Molecule(366 atoms) at 139700365342352> <Molecule(30 atoms) at 139700354190096>


In [9]:
cg = spd.Spinner(
    step_size=0.5,
    rotation_step_size=5,
    num_conformers=100,
    max_attempts=2000,
)

In [10]:
supramolecule = spd.SupraMolecule.init_from_components(components=(host, guest))

In [11]:
energies = {}
for conformer in cg.get_conformers(supramolecule):
    print(conformer)
    print(conformer.get_cid(), conformer.get_potential())
    for i, comp in enumerate(list(conformer.get_components())):
        if i == 0:
            cage1 = cage1.with_position_matrix(comp.get_position_matrix())
        elif i == 1:
            stk_guest = stk_guest.with_position_matrix(comp.get_position_matrix())

    complex_mol = stk.ConstructedMolecule(
        topology_graph=stk.host_guest.Complex(
            host=stk.BuildingBlock.init_from_molecule(cage1), 
            guests=stk.host_guest.Guest(stk_guest),
        )
    )
    complex_mol.write(
        f'hg_example_output/hg_conf_{conformer.get_cid()}.mol'
    )
    energies[conformer.get_cid()] = stko.UFFEnergy(ignore_inter_interactions=False).get_energy(complex_mol)

SupraMolecule(2 components, <Molecule(366 atoms) at 139700365342352>, <Molecule(30 atoms) at 139700354190096>)
0 -2.7470047907899104
SupraMolecule(2 components, <Molecule(366 atoms) at 139700365342352>, <Molecule(30 atoms) at 139700319040912>)
1 -2.2406527148811453
SupraMolecule(2 components, <Molecule(366 atoms) at 139700365342352>, <Molecule(30 atoms) at 139700353946000>)
2 -3.2671732112684144
SupraMolecule(2 components, <Molecule(366 atoms) at 139700365342352>, <Molecule(30 atoms) at 139700318851728>)
3 -5.045257999598347
SupraMolecule(2 components, <Molecule(366 atoms) at 139700365342352>, <Molecule(30 atoms) at 139700318851856>)
4 -3.5192538903850936
SupraMolecule(2 components, <Molecule(366 atoms) at 139700365342352>, <Molecule(30 atoms) at 139700353951632>)
5 -3.731675430276508


[22:09:47] UFFTYPER: Unrecognized atom type: Pd6+2 (0)
[22:09:47] UFFTYPER: Unrecognized atom type: Pd6+2 (1)
[22:09:47] UFFTYPER: Unrecognized atom type: Pd6+2 (2)
[22:09:47] UFFTYPER: Unrecognized atom type: Pd6+2 (3)
[22:09:47] UFFTYPER: Unrecognized atom type: Pd6+2 (4)
[22:09:47] UFFTYPER: Unrecognized atom type: Pd6+2 (5)
[22:09:47] UFFTYPER: Unrecognized atom type: Pd6+2 (0)
[22:09:47] UFFTYPER: Unrecognized atom type: Pd6+2 (1)
[22:09:47] UFFTYPER: Unrecognized atom type: Pd6+2 (2)
[22:09:47] UFFTYPER: Unrecognized atom type: Pd6+2 (3)
[22:09:47] UFFTYPER: Unrecognized atom type: Pd6+2 (4)
[22:09:47] UFFTYPER: Unrecognized atom type: Pd6+2 (5)
[22:09:47] UFFTYPER: Unrecognized atom type: Pd6+2 (0)
[22:09:47] UFFTYPER: Unrecognized atom type: Pd6+2 (1)
[22:09:47] UFFTYPER: Unrecognized atom type: Pd6+2 (2)
[22:09:47] UFFTYPER: Unrecognized atom type: Pd6+2 (3)
[22:09:47] UFFTYPER: Unrecognized atom type: Pd6+2 (4)
[22:09:47] UFFTYPER: Unrecognized atom type: Pd6+2 (5)
[22:09:47]

SupraMolecule(2 components, <Molecule(366 atoms) at 139700365342352>, <Molecule(30 atoms) at 139700353947728>)
6 -7.256757393232636
SupraMolecule(2 components, <Molecule(366 atoms) at 139700365342352>, <Molecule(30 atoms) at 139700318851536>)
7 -10.050725840062682
SupraMolecule(2 components, <Molecule(366 atoms) at 139700365342352>, <Molecule(30 atoms) at 139700318987152>)
8 -13.416424571883299


[22:09:48] UFFTYPER: Unrecognized atom type: Pd6+2 (0)
[22:09:48] UFFTYPER: Unrecognized atom type: Pd6+2 (1)
[22:09:48] UFFTYPER: Unrecognized atom type: Pd6+2 (2)
[22:09:48] UFFTYPER: Unrecognized atom type: Pd6+2 (3)
[22:09:48] UFFTYPER: Unrecognized atom type: Pd6+2 (4)
[22:09:48] UFFTYPER: Unrecognized atom type: Pd6+2 (5)
[22:09:48] UFFTYPER: Unrecognized atom type: Pd6+2 (0)
[22:09:48] UFFTYPER: Unrecognized atom type: Pd6+2 (1)
[22:09:48] UFFTYPER: Unrecognized atom type: Pd6+2 (2)
[22:09:48] UFFTYPER: Unrecognized atom type: Pd6+2 (3)
[22:09:48] UFFTYPER: Unrecognized atom type: Pd6+2 (4)
[22:09:48] UFFTYPER: Unrecognized atom type: Pd6+2 (5)
[22:09:48] UFFTYPER: Unrecognized atom type: Pd6+2 (0)
[22:09:48] UFFTYPER: Unrecognized atom type: Pd6+2 (1)
[22:09:48] UFFTYPER: Unrecognized atom type: Pd6+2 (2)
[22:09:48] UFFTYPER: Unrecognized atom type: Pd6+2 (3)
[22:09:48] UFFTYPER: Unrecognized atom type: Pd6+2 (4)
[22:09:48] UFFTYPER: Unrecognized atom type: Pd6+2 (5)


SupraMolecule(2 components, <Molecule(366 atoms) at 139700365342352>, <Molecule(30 atoms) at 139700318986000>)
9 -12.187077975101005
SupraMolecule(2 components, <Molecule(366 atoms) at 139700365342352>, <Molecule(30 atoms) at 139700319305552>)
10 -11.217497709287136
SupraMolecule(2 components, <Molecule(366 atoms) at 139700365342352>, <Molecule(30 atoms) at 139700319305232>)
11 -13.389118352771305


...

[22:09:51] UFFTYPER: Unrecognized atom type: Pd6+2 (0)
[22:09:51] UFFTYPER: Unrecognized atom type: Pd6+2 (1)
[22:09:51] UFFTYPER: Unrecognized atom type: Pd6+2 (2)
[22:09:51] UFFTYPER: Unrecognized atom type: Pd6+2 (3)
[22:09:51] UFFTYPER: Unrecognized atom type: Pd6+2 (4)
[22:09:51] UFFTYPER: Unrecognized atom type: Pd6+2 (5)


SupraMolecule(2 components, <Molecule(366 atoms) at 139700365342352>, <Molecule(30 atoms) at 139700319228432>)
24 -29.433503833383295


[22:09:52] UFFTYPER: Unrecognized atom type: Pd6+2 (0)
[22:09:52] UFFTYPER: Unrecognized atom type: Pd6+2 (1)
[22:09:52] UFFTYPER: Unrecognized atom type: Pd6+2 (2)
[22:09:52] UFFTYPER: Unrecognized atom type: Pd6+2 (3)
[22:09:52] UFFTYPER: Unrecognized atom type: Pd6+2 (4)
[22:09:52] UFFTYPER: Unrecognized atom type: Pd6+2 (5)


SupraMolecule(2 components, <Molecule(366 atoms) at 139700365342352>, <Molecule(30 atoms) at 139700319105808>)
25 -30.94481183814383


[22:09:53] UFFTYPER: Unrecognized atom type: Pd6+2 (0)
[22:09:53] UFFTYPER: Unrecognized atom type: Pd6+2 (1)
[22:09:53] UFFTYPER: Unrecognized atom type: Pd6+2 (2)
[22:09:53] UFFTYPER: Unrecognized atom type: Pd6+2 (3)
[22:09:53] UFFTYPER: Unrecognized atom type: Pd6+2 (4)
[22:09:53] UFFTYPER: Unrecognized atom type: Pd6+2 (5)


In [12]:
energies

{0: 7998.783057555388,
 1: 7466.138002176434,
 2: 7486.749336291635,
 3: 7471.304587467206,
 4: 7470.30805315125,
 5: 7466.28253511995,
 6: 7465.344865981112,
 7: 7470.867476789871,
 8: 7465.763725061799,
 9: 7465.948764752859,
 10: 7466.127781588913,
 11: 7465.736417196755,
 12: 7465.50474073223,
 13: 7472.038277986004,
 14: 7468.478668757239,
 15: 7466.770373450627,
 16: 7465.996190475216,
 17: 7467.695081531071,
 18: 7469.769579949688,
 19: 7521.8601589746395,
 20: 7511.341691454011,
 21: 7613.762517431874,
 22: 7465.849219568273,
 23: 7466.102228806958,
 24: 7466.0912564498285,
 25: 7466.076819770992}